In [2]:
import os
import mlflow
import mlflow.sklearn

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# silent warnings
import warnings
warnings.filterwarnings("ignore")
# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
# import os
# import mlflow
# import pandas as pd
# from datetime import datetime
# import uuid
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.pipeline import Pipeline

# class TrainingSetUp:
#     def __init__(self, *args, **kwargs):
#         self.id = kwargs.get('id', str(uuid.uuid4()))
#         self.created_at = kwargs.get('created_at', datetime.now())
#         self.updated_at = kwargs.get('updated_at', datetime.now())
        
#         if isinstance(self.created_at, str):
#             self.created_at = datetime.strptime(self.created_at, '%Y-%m-%d %H:%M:%S')
        
#         for key, value in kwargs.items():
#             setattr(self, key, value)

#         self.X_train = self.X_test = self.y_train = self.y_test = None
#         self.y_pred = self.y_pred_proba = self.y_pred_proba_train = None
#         self.y_pred_train = self.y_pred_proba_test = self.y_pred_test = None
#         self.__pipeline = self.__model = self.__grid_search = None

#     def load_data(self):
#         self.data = pd.read_csv(self.data_path)
#         self.X = self.data.drop('target', axis=1)
#         self.y = self.data['target']

#     def split_data(self, test_size=0.2, random_state=42):
#         self.X_train, self.X_test, self.y_train, self.y_test = \
#             train_test_split(self.X, self.y, test_size=test_size, random_state=random_state)

#     def pipeline(self):
#         if hasattr(self, 'scalers') and hasattr(self, 'model'):
#             self.__pipeline = Pipeline([
#                 ('scaler', self.scalers),
#                 ('model', self.model)
#             ])
#         else:
#             raise AttributeError('scalers and model must be defined')

#     def grid_search(self):
#         if self.__pipeline is None:
#             self.pipeline()
#         requires = ['param_grid', 'cv', 'scoring']
#         if all(hasattr(self, k) for k in requires):
#             self.__grid_search = GridSearchCV(self.__pipeline,
#                                               self.param_grid,
#                                               cv=self.cv,
#                                               scoring=self.scoring)
#         else:
#             missing = [k for k in requires if not hasattr(self, k)]
#             raise AttributeError(f'{", ".join(missing)} must be defined')

# class TrainModel(TrainingSetUp):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.__mlflow_uri = None
#         self.__mlflow_metaData = None
#         mlflow_info = kwargs.get('mlflow', {})
#         self.setup_mlflow(mlflow_info.get('exp_name'), mlflow_info.get('artifact_uri'))
#         data_path = kwargs.get('data_path')
#         if data_path:
#             self.load_data(data_path)

#     def setup_mlflow(self, exp_name=None, artifact_uri=None):
#         if exp_name is None:
#             exp_name = f'mlflow_{self.id}'
        
#         os.makedirs("./mlruns", exist_ok=True)
#         default_uri = f'./mlruns/{exp_name}'
#         tracking_uri = os.getenv('MLFLOW_TRACKING_URI_TRAIN', default_uri)
#         mlflow.set_tracking_uri(tracking_uri)
#         mlflow.set_experiment(exp_name)
#         self.__mlflow_uri = tracking_uri

#         if artifact_uri:
#             mlflow.set_artifact_uri(artifact_uri)

#     def train(self):
#         with mlflow.start_run():
#             if self.__grid_search is None:
#                 self.grid_search()
            
#             self.__grid_search.fit(self.X_train, self.y_train)
#             self.__model = self.__grid_search.best_estimator_

#             # Log parameters
#             mlflow.log_params(self.__grid_search.best_params_)

#             # Log metrics
#             mlflow.log_metric("best_score", self.__grid_search.best_score_)

#             # Log model
#             mlflow.sklearn.log_model(self.__model, "model")

#             # Make predictions
#             self.y_pred = self.__model.predict(self.X_test)
#             self.y_pred_proba = self.__model.predict_proba(self.X_test)

#             # Log additional metrics (you can add more as needed)
#             from sklearn.metrics import accuracy_score, f1_score
#             accuracy = accuracy_score(self.y_test, self.y_pred)
#             f1 = f1_score(self.y_test, self.y_pred, average='weighted')
#             mlflow.log_metric("accuracy", accuracy)
#             mlflow.log_metric("f1_score", f1)

#     def evaluate(self):
#         # Add your evaluation logic here
#         pass

#     def save_model(self, path):
#         if self.__model is None:
#             raise Exception("Model has not been trained yet")
#         mlflow.sklearn.save_model(self.__model, path)

#     def load_model(self, path):
#         self.__model = mlflow.sklearn.load_model(path)

# if __name__ == "__main__":
#     # Example usage
#     kwargs = {
#         'mlflow': {'exp_name': 'my_experiment'},
#         'data_path': 'path/to/your/data.csv'
#     }

#     trainer = TrainModel(**kwargs)
#     trainer.load_data()
#     trainer.split_data()
#     trainer.scalers = StandardScaler()
#     trainer.model = RandomForestClassifier()
#     trainer.param_grid = {
#         'model__n_estimators': [100, 200, 300],
#         'model__max_depth': [5, 10, None]
#     }
#     trainer.cv = 5
#     trainer.scoring = 'accuracy'
#     trainer.train()
#     trainer.evaluate()
#     trainer.save_model('path/to/save/model')

In [36]:
import sys
sys.path.append("..")
from utils.model_func import load_config
import logging
from ml.training import TrainModel
from utils.model_func import load_config

CONFIG_PATH = 'config.json'

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Usage
if __name__ == "__main__":
    logging.info("Starting the training process")

    # Load configuration
    
    config = load_config(CONFIG_PATH)['MODEL1']
    config.update({k: eval(v) if (k == 'scalers' or k=='model') \
                   else v  for k, v in config.items()})

    trainer = TrainModel(config.get('MODEL1'))
    
    trainer.load_data(trainer.data_path)
    trainer.split_data()
    
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    
    trainer.scalers = StandardScaler()
    trainer.model = RandomForestClassifier()
    trainer.param_grid = {
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [5, 10, None]
    }
    trainer.cv = 5
    trainer.scoring = 'accuracy'
    
    trainer.train()
    trainer.evaluate()
    trainer.save_model('path/to/save/model')
    
    logging.info("Training process completed")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\lenovo\anaconda3\envs\amsVenv\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_9172\3077490805.py", line 5, in <module>
    from ml.training import TrainModel
  File "e:\amihere\programmingLessons\ALX-Backend\FINAL PROJECT\TripNYC\models\ml\ml.py", line 7, in <module>
    from base.base_model import BaseModel, Base
  File "e:\amihere\programmingLessons\ALX-Backend\FINAL PROJECT\TripNYC\models\ml\..\base\__init__.py", line 3, in <module>
    from models.location.borough import Borough
ModuleNotFoundError: No module named 'models'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lenovo\anaconda3\envs\amsVenv\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceb

In [34]:
import sys
sys.path.append("..")

from utils.model_func import load_config

config = load_config('../utils/config/ml_config.json')['MODEL1']
print(config)
config.update({k: eval(v) if (k == 'scalers' or k=='model') else v  for k, v in config.items()})
print(config)

{'mlflow': {'exp_name': 'my_experiment'}, 'data_path': 'path/to/your/data.csv', 'scalers': 'StandardScaler()', 'model': 'RandomForestClassifier()', 'param_grid': {'model__n_estimators': [100, 200, 300], 'model__max_depth': [5, 10]}, 'cv': 5, 'scoring': 'accuracy', 'model_path': 'path/to/save/model'}
{'mlflow': {'exp_name': 'my_experiment'}, 'data_path': 'path/to/your/data.csv', 'scalers': StandardScaler(), 'model': RandomForestClassifier(), 'param_grid': {'model__n_estimators': [100, 200, 300], 'model__max_depth': [5, 10]}, 'cv': 5, 'scoring': 'accuracy', 'model_path': 'path/to/save/model'}


In [ ]:
# Create a random dataset
X = np.random.rand(100, 4)
y = np.random.rand(100)

# Define a pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Define a grid search
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10, None],
    'classifier__min_samples_split': [2, 5, 10],
    '__name__': ['RandomForestClassifier_1']
}

# Create a grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                           scoring='accuracy', n_jobs=-1)

# Set the experiment name
mlflow.set_experiment("RandomForest_GridSearch")
# Set the MLflow tracking URI
tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
if tracking_uri is None:
    os.makedirs("./mlruns", exist_ok=True)
    tracking_uri = "./mlruns"
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))



# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 5)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Train model (simplified)
    model = RandomForestClassifier(n_estimators=100, max_depth=5)
    model.fit(X_train, y_train)

    # Log metrics
    accuracy = accuracy_score(y_test, model.predict(X_test))
    mlflow.log_metric("accuracy", accuracy)

    # Log model
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Log a plot as an artifact
    plt.figure()
    plt.plot(model.feature_importances_)
    plt.title("Feature Importances")
    plt.savefig("feature_importances.png")
    mlflow.log_artifact("feature_importances.png")

    # Log a tag
    mlflow.set_tag("model_type", "random_forest")

    # Log a custom artifact (e.g., feature names)
    feature_names = ["feature1", "feature2", "feature3"]
    np.savetxt("feature_names.txt", feature_names, fmt='%s')
    mlflow.log_artifact("feature_names.txt")

In [39]:
from sys import platform
import time
import GPUtil
import platform
import mlflow
gpus = GPUtil.getGPUs()
x = 0
print(len(gpus))
for gpu in gpus:
    print(f"Name: {gpu.name}")
    print(f"Total Memory: {gpu.memoryTotal}MB")
    print(f"Used Memory: {gpu.memoryUtil * gpu.memoryTotal / 100}MB")
    print(f"Driver Version: {gpu.__dict__}")
    print(f"UUID: {gpu.uuid}")
    x += 1
    print(x)


# import psutil
# while True:
#     print(psutil.virtual_memory().percent)
#     time.sleep(10)

eval("mlflow.__version__")


1
Name: GeForce GTX 1650 Ti
Total Memory: 4096.0MB
Used Memory: 22.78MB
Driver Version: {'id': 0, 'uuid': 'GPU-48f08f36-9b67-ccde-313a-b9b3d548e507', 'load': 0.17, 'memoryUtil': 0.55615234375, 'memoryTotal': 4096.0, 'memoryUsed': 2278.0, 'memoryFree': 1818.0, 'driver': '457.49', 'name': 'GeForce GTX 1650 Ti', 'serial': '[N/A]', 'display_mode': 'Enabled', 'display_active': 'Enabled', 'temperature': 46.0}
UUID: GPU-48f08f36-9b67-ccde-313a-b9b3d548e507
1


'2.14.2'

In [76]:
# import sys
# sys.path.append("..")

x = 'train_model.py'

# eval("os.path.getsize('models/ml/.pkl')")
import os

eval("os.environ.get('MLFLOW_TRACKING_URI', 'http://localhost:5000')")

'http://localhost:5000'

In [5]:
#Yellow Taxi Data
import sys
df_yellow = []
base = "../../../Data/yellow/{}/yellow_tripdata_{}-{}.csv"

yrs = [(2024, 4), (2023, 6), (2022, 6), (2021, 6), (2020, 6)]


for yr, mth in yrs:
    y2024 = []
    print(yr)
    for i in range(1, mth+1):

        df = pd.read_csv(base.format(yr, yr, str(i).zfill(2)))
        # take 50% of the data
        df = df.sample(frac=0.5)
        print(f"\t{yr}-{i}-{df.shape}")
        print(f"\t\t{list(df.columns)}")
        y2024.append(df)
        df = None
    df_yellow.append(pd.concat(y2024))
    # clear buffer
    y2024 = []
    # clear stdout
    sys.stdout.flush()

2024
	2024-1-(1482312, 19)
		['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']
	2024-2-(1503763, 19)
		['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']
	2024-3-(1791314, 19)
		['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge

In [6]:
df = pd.concat([_.sample(frac=0.3) for _ in df_yellow])

In [7]:
date_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

date_cols_rename = ['pickup_datetime', 'dropoff_datetime', ]
df.rename(columns={k: v for k, v in zip(date_cols, date_cols_rename)}, inplace=True)

new_cols = ['pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time']

for col, new_col in zip(date_cols_rename, new_cols):
    df[new_col] = df[col].dt.date if 'date' in new_col else df[col].dt.time

new_name = ['pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12211171 entries, 423949 to 6267508
Data columns (total 22 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               float64       
 1   pickup_datetime        datetime64[ns]
 2   dropoff_datetime       datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           float64       
 8   DOLocationID           float64       
 9   payment_type           float64       
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee          

In [8]:
df.head()

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,airport_fee,pickup_date,pickup_time
423949,2.0,2024-03-05 12:06:35,2024-03-05 12:17:27,2.0,2.60,1.0,N,140.0,137.0,1.0,...,0.5,3.78,0.00,1.0,22.68,2.5,0.00,NaN,2024-03-05,12:17:27
1351186,2.0,2024-02-14 22:57:02,2024-02-14 23:32:57,3.0,19.46,2.0,N,132.0,238.0,1.0,...,0.5,16.19,6.94,1.0,98.88,2.5,1.75,NaN,2024-02-14,23:32:57
2140301,1.0,2024-02-23 10:28:02,2024-02-23 10:41:11,1.0,1.80,1.0,N,140.0,162.0,1.0,...,0.5,3.50,0.00,1.0,21.00,2.5,0.00,NaN,2024-02-23,10:41:11
1509366,1.0,2024-02-16 14:13:37,2024-02-16 14:18:56,2.0,0.90,1.0,N,142.0,50.0,2.0,...,0.5,0.00,0.00,1.0,11.20,2.5,0.00,NaN,2024-02-16,14:18:56
2693020,2.0,2024-01-30 18:46:20,2024-01-30 18:49:11,1.0,0.55,1.0,N,236.0,75.0,2.0,...,0.5,0.00,0.00,1.0,11.60,2.5,0.00,NaN,2024-01-30,18:49:11


In [9]:
def check_unique(d):
    for c in d.columns:
        print(f"{c}: {d[c].nunique()}")
def check_missing(d):
    for c in d.columns:
        print(f"{c}: {d[c].isnull().sum()} ({d[c].isnull().sum()/d.shape[0] * 100:.2f}%)")

In [10]:
# Check for unique values
check_unique(df)

VendorID: 4
pickup_datetime: 10685137
dropoff_datetime: 10680658
passenger_count: 10
trip_distance: 6506
RatecodeID: 7
store_and_fwd_flag: 2
PULocationID: 262
DOLocationID: 261
payment_type: 6
fare_amount: 11591
extra: 229
mta_tax: 24
tip_amount: 5329
tolls_amount: 2003
improvement_surcharge: 5
total_amount: 23593
congestion_surcharge: 7
Airport_fee: 5
airport_fee: 3
pickup_date: 877
pickup_time: 86400


In [11]:
# Check for missing values
check_missing(df)

VendorID: 0 (0.00%)
pickup_datetime: 0 (0.00%)
dropoff_datetime: 0 (0.00%)
passenger_count: 500980 (4.10%)
trip_distance: 0 (0.00%)
RatecodeID: 500980 (4.10%)
store_and_fwd_flag: 500980 (4.10%)
PULocationID: 0 (0.00%)
DOLocationID: 0 (0.00%)
payment_type: 0 (0.00%)
fare_amount: 0 (0.00%)
extra: 0 (0.00%)
mta_tax: 0 (0.00%)
tip_amount: 0 (0.00%)
tolls_amount: 0 (0.00%)
improvement_surcharge: 0 (0.00%)
total_amount: 0 (0.00%)
congestion_surcharge: 500980 (4.10%)
Airport_fee: 8028556 (65.75%)
airport_fee: 7792719 (63.82%)
pickup_date: 0 (0.00%)
pickup_time: 0 (0.00%)


In [12]:
df_num = df.select_dtypes(include=['number'])
def_non_num = df.select_dtypes(exclude=['number'])
num_cols = df_num.columns.to_list()
non_num_cols = def_non_num.columns.to_list()

print(f"All columns: {df.shape}")
print(f"Numerical columns: {df_num.shape}")
print(f"Non-numerical columns: {def_non_num.shape}")

All columns: (12211171, 22)
Numerical columns: (12211171, 17)
Non-numerical columns: (12211171, 5)


### Process Numeric Cols

In [13]:
df_num.head()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,airport_fee
423949,2.0,2.0,2.60,1.0,140.0,137.0,1.0,14.9,0.0,0.5,3.78,0.00,1.0,22.68,2.5,0.00,NaN
1351186,2.0,3.0,19.46,2.0,132.0,238.0,1.0,70.0,0.0,0.5,16.19,6.94,1.0,98.88,2.5,1.75,NaN
2140301,1.0,1.0,1.80,1.0,140.0,162.0,1.0,13.5,2.5,0.5,3.50,0.00,1.0,21.00,2.5,0.00,NaN
1509366,1.0,2.0,0.90,1.0,142.0,50.0,2.0,7.2,2.5,0.5,0.00,0.00,1.0,11.20,2.5,0.00,NaN
2693020,2.0,1.0,0.55,1.0,236.0,75.0,2.0,5.1,2.5,0.5,0.00,0.00,1.0,11.60,2.5,0.00,NaN


In [14]:
check_unique(df_num)

VendorID: 4
passenger_count: 10
trip_distance: 6506
RatecodeID: 7
PULocationID: 262
DOLocationID: 261
payment_type: 6
fare_amount: 11591
extra: 229
mta_tax: 24
tip_amount: 5329
tolls_amount: 2003
improvement_surcharge: 5
total_amount: 23593
congestion_surcharge: 7
Airport_fee: 5
airport_fee: 3


In [38]:
check_missing(df_num)

VendorID: 0 (0.00%)
passenger_count: 502572 (4.12%)
trip_distance: 0 (0.00%)
RatecodeID: 502572 (4.12%)
PULocationID: 0 (0.00%)
DOLocationID: 0 (0.00%)
payment_type: 0 (0.00%)
fare_amount: 0 (0.00%)
extra: 0 (0.00%)
mta_tax: 0 (0.00%)
tip_amount: 0 (0.00%)
tolls_amount: 0 (0.00%)
improvement_surcharge: 0 (0.00%)
total_amount: 0 (0.00%)
congestion_surcharge: 502572 (4.12%)
Airport_fee: 8029286 (65.75%)
airport_fee: 7793333 (63.82%)


In [ ]:
cols_fill = ['passenger_count', 'congestion_surcharge']
cols_drop = ['Airport_fee', 'airport_fee']
cols_to_cat = ['VendorID', 'RatecodeID', 'payment_type', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID']
